In [13]:
import pandas as pd
import swisseph as swe
from vedicastro.utils import *
from vedicastro import VedicAstro as va

In [14]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [15]:
def decimal_to_dms(decimal_deg: float):
    signs = ['Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo', 'Libra', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces']
    degrees = int(decimal_deg)
    sign_index = degrees // 30
    degrees_in_sign = degrees % 30
    minutes = int((decimal_deg - degrees) * 60)
    seconds = (decimal_deg - degrees - minutes/60) * 3600.00
    return signs[sign_index], f"{degrees_in_sign:+02}:{minutes:02}:{seconds:02.0f}"

def get_rl_nl_sl_data(deg : float):
    """
    Returns the  Rashi (Sign) Lord, Nakshatra, Nakshatra Pada, Nakshatra Lord, Sub Lord and Sub Sub Lord 
    corresponding to the given degree.
    """
    from vedicastro.VedicAstro import NAKSHATRAS, SIGN_LORDS
    duration = [7, 20, 6, 10, 7, 18, 16, 19, 17]

    lords = ["Ketu", "Venus", "Sun", "Moon", "Mars", "Rahu", "Jupiter", "Saturn", "Mercury"]

    star_lords = lords * 3 ## lords for the 27 Nakshatras

    ## Compute Sign lords
    sign_deg = deg % 360  # Normalize degree to [0, 360)
    sign_index = int(sign_deg // 30)  # Each zodiac sign is 30 degrees
    
    # Compute Nakshatra details
    nakshatra_deg = sign_deg % 13.332  # Each nakshatra is 13.332 degrees
    nakshatra_index = int(sign_deg // 13.332)  # Find the nakshatra index
    pada = int((nakshatra_deg % 13.332) // 3.325) + 1  # Each pada is 3.325 degrees

    # Ensure nakshatra_index is within bounds
    nakshatra_index = nakshatra_index % len(NAKSHATRAS)        

    # Compute SubLords
    deg = deg - 120 * int(deg / 120)
    degcum = 0
    i = 0

    while i < 9:
        deg_nl = 360 / 27
        j = i
        while True:
            deg_sl = deg_nl * duration[j] / 120
            k = j
            while True:
                deg_ss = deg_sl * duration[k] / 120
                degcum += deg_ss
                if degcum >= deg:
                    return {"Nakshatra": NAKSHATRAS[nakshatra_index] + "-" + str(pada), 
                            "RasiLord": SIGN_LORDS[sign_index], "NakshatraLord": star_lords[nakshatra_index],
                            "SubLord": lords[j], "SubSubLord": lords[k] }
                k = (k + 1) % 9
                if k == j:
                    break
            j = (j + 1) % 9
            if j == i:
                break
        i += 1  

### Common `SideReal` Ayanamsas for Vedic Astrology
- https://astrorigin.com/pyswisseph/sphinx/ephemerides/sidereal/krishnamurti_ayanamshas.html
- https://astrorigin.com/pyswisseph/sphinx/ephemerides/sidereal/spica_citra_tradition_and_lahiri_ayanamsha.html


In [16]:
SWE_AYANAMSHAS = { "Krishnamurti_Old" : swe.SIDM_KRISHNAMURTI, "Krishnamurti_New": swe.SIDM_KRISHNAMURTI_VP291,
                "Lahiri_1940": swe.SIDM_LAHIRI_1940, "Lahiri_VP285" : swe.SIDM_LAHIRI_VP285, 
                "Lahiri_ICRC" : swe.SIDM_LAHIRI_ICRC,
                }

In [17]:
## Case 1
# year = 2024
# month = 8
# day = 14
# hour = 22
# minute = 31
# secs = 31
# latitude, longitude, utc = 28.6334, 77.2834, "+5:30" ## Delhi

In [18]:
## Case 2
year = 2000
month = 12
day = 30
hour = 22
minute = 31
secs = 59
latitude, longitude, utc_offset = 28.6334, 77.2834, "+5:30" ## Delhi

In [19]:
from pytz import timezone
zone = "Asia/Calcutta"
### TIMEZONE INFORMATION
tz = timezone(zone)
localized_time = tz.localize(datetime(year, month, day, hour, minute, secs))

### `Asc` Results using native `pyswisseph` library


In [20]:
utc_float =  utc_offset_str_to_float(utc_offset)

utc = swe.utc_time_zone(year, month, day, hour = hour, minutes = minute, seconds = secs, offset = utc_float)
_ , jd_start = swe.utc_to_jd(*utc) ## Unpacks utc tuple
current_time = jd_start

In [21]:
# Assuming SWE_AYANAMSHAS and other necessary imports and variables are already defined

results = []  # List to store the results

for ayan in list(SWE_AYANAMSHAS.keys()):
    swe.set_sid_mode(SWE_AYANAMSHAS.get(ayan))  # Set the ayanamsa based on the current key
    cusps, asmc = swe.houses_ex(current_time, latitude, longitude, b'P', flags=swe.FLG_SIDEREAL)
    ## Compute results for only Ascendant
    # asc = cusps[0]
    # sign, sign_dms = decimal_to_dms(asc)
    # res = { 'Ayanamsa': ayan, 'AscSign': sign, 'AscDecLong': asc, 'AscDMS': sign_dms}
    # res.update(get_rl_nl_sl_data(deg = asc))
    # results.append(res)

    ## Compute data for all cusps
    for idx, cusp in zip(range(1, len(cusps)+1),cusps):
        sign, sign_dms = decimal_to_dms(cusp)
        res = {'Ayanamsa': ayan, "HouseID" : idx,'DecLong': cusp, 'Sign': sign,'SignDMS': sign_dms}
        res.update(get_rl_nl_sl_data(deg = cusp))
        results.append(res)


# Convert the list of dictionaries into a pandas DataFrame
df_swisseph = pd.DataFrame(results)
df_swisseph

,Ayanamsa,HouseID,DecLong,Sign,SignDMS,Nakshatra,RasiLord,NakshatraLord,SubLord,SubSubLord
0,Krishnamurti_Old,1,140.590161,Leo,+20:35:25,PūrvaPhalgunī-3,Sun,Venus,Jupiter,Saturn
1,Krishnamurti_Old,2,167.716710,Virgo,+17:43:00,Hasta-3,Mercury,Moon,Saturn,Jupiter
2,Krishnamurti_Old,3,198.008786,Libra,+18:00:32,Svati-4,Venus,Rahu,Sun,Venus
3,Krishnamurti_Old,4,229.833947,Scorpio,+19:50:02,Jyeshtha-1,Mars,Mercury,Venus,Moon
4,Krishnamurti_Old,5,261.517184,Sagittarius,+21:31:02,PurvaAshadha-3,Jupiter,Venus,Jupiter,Moon
5,Krishnamurti_Old,6,292.032823,Capricorn,+22:01:58,Shravana-4,Saturn,Moon,Venus,Saturn
6,Krishnamurti_Old,7,320.590161,Aquarius,+20:35:25,PurvaBhādrapadā-1,Saturn,Jupiter,Jupiter,Mercury
7,Krishnamurti_Old,8,347.716710,Pisces,+17:43:00,Revati-1,Jupiter,Mercury,Mercury,Mars
8,Krishnamurti_Old,9,18.008786,Aries,+18:00:32,Bharani-2,Mars,Venus,Mars,Sun
9,Krishnamurti_Old,10,49.833947,Taurus,+19:50:02,Rohini-3,Venus,Moon,Ketu,Venus


### `Asc` Results from `VedicAstro` which uses `flatlib`

In [22]:
from flatlib import const

flatlib_ayanamshas =  ["Krishnamurti", "Lahiri"]
house_system = "Placidus"

flatlib_results = []
for ayan in flatlib_ayanamshas:
    vhd = va.VedicHoroscopeData(year = year, month = month, day = day, hour = hour, minute = minute, second = secs, utc = utc_offset, 
                            latitude = latitude, longitude = longitude, ayanamsa = ayan, house_system = house_system)
    chart = vhd.generate_chart()

    ## Compute results for only the Ascendant
    # asc = chart.get(const.ASC)
    # asc_sign, asc_dms = str(asc).split()[1:]
    # res = {"Ayanamsa" : ayan, "AscSign": asc.sign, "AscDecLong": asc.lon, "AscDMS" : asc_dms[:-1]}
    # res.update(get_rl_nl_sl_data(deg = asc.lon))
    # flatlib_results.append(res)

    for house in chart.houses:
        _, house_sign, house_dms, _ = str(house).split()
        res = {"Ayanamsa" : ayan, "HouseID": house.id,"AscSign": house_sign, "AscDecLong": house.lon, "AscDMS" : house_dms[:-1]}
        res.update(get_rl_nl_sl_data(deg = house.lon))
        flatlib_results.append(res)
        

In [23]:
df_flatlib = pd.DataFrame(flatlib_results)

In [24]:
# Convert DataFrames to HTML and concatenate them with some space in between
df1_html = df_swisseph.to_html()
df2_html = df_flatlib.to_html()

# df1_html = df_swisseph[df_swisseph["Ayanamsa"] == "Krishnamurti_New"].to_html()
# df2_html = df_flatlib[df_flatlib["Ayanamsa"] == "Krishnamurti"].to_html()

# Titles for each DataFrame
title1 = "Swisseph - Asc Results"
title2 = "Flatlib - Asc Results"

# Display them side by side with titles using HTML display with formatted string
html = f"""
<div style='display:flex; justify-content:space-between;'>
    <div style='width: 45%;'>
        <h2>{title1}</h2>  <!-- Title for the first DataFrame -->
        {df1_html}
    </div>
    <div style='width: 45%;'>
        <h2>{title2}</h2>  <!-- Title for the second DataFrame -->
        {df2_html}
    </div>
</div>
"""
print(f"Asc Results for {localized_time} at Lat:{latitude} | Long: {longitude}")
display(HTML(html))

Asc Results for 2000-12-30 22:31:59+05:30 at Lat:28.6334 | Long: 77.2834


,Ayanamsa,HouseID,DecLong,Sign,SignDMS,Nakshatra,RasiLord,NakshatraLord,SubLord,SubSubLord
0,Krishnamurti_Old,1,140.590161,Leo,+20:35:25,PūrvaPhalgunī-3,Sun,Venus,Jupiter,Saturn
1,Krishnamurti_Old,2,167.716710,Virgo,+17:43:00,Hasta-3,Mercury,Moon,Saturn,Jupiter
2,Krishnamurti_Old,3,198.008786,Libra,+18:00:32,Svati-4,Venus,Rahu,Sun,Venus
3,Krishnamurti_Old,4,229.833947,Scorpio,+19:50:02,Jyeshtha-1,Mars,Mercury,Venus,Moon
4,Krishnamurti_Old,5,261.517184,Sagittarius,+21:31:02,PurvaAshadha-3,Jupiter,Venus,Jupiter,Moon
5,Krishnamurti_Old,6,292.032823,Capricorn,+22:01:58,Shravana-4,Saturn,Moon,Venus,Saturn
6,Krishnamurti_Old,7,320.590161,Aquarius,+20:35:25,PurvaBhādrapadā-1,Saturn,Jupiter,Jupiter,Mercury
7,Krishnamurti_Old,8,347.716710,Pisces,+17:43:00,Revati-1,Jupiter,Mercury,Mercury,Mars
8,Krishnamurti_Old,9,18.008786,Aries,+18:00:32,Bharani-2,Mars,Venus,Mars,Sun
9,Krishnamurti_Old,10,49.833947,Taurus,+19:50:02,Rohini-3,Venus,Moon,Ketu,Venus
